# Setup

- typing
- documentation and comments
- test cases
- requirements and versioning

In [1]:
import requests
import pandas
import pendulum
from pendulum import today, Date
from pathlib import Path
from typing import Union
import zipfile
import tempfile
import os
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import logging
from requests.exceptions import HTTPError, ReadTimeout, Timeout
import glob
from copy import copy

# import scrapy
# from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# from webdriver_manager import logger
from selenium.webdriver.remote.remote_connection import LOGGER as seleniumLogger

from IPython.core.interactiveshell import InteractiveShell
from tqdm import tqdm

# from pandarallel import pandarallel

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
tqdm.pandas()
# pandarallel.initialize(nb_workers=4)

pd.set_option("display.max_columns", None)

InteractiveShell.ast_node_interactivity = "all"
# %reload_ext lab_black

# logging.getLogger().setLevel(logging.INFO)
os.environ["WDM_LOG_LEVEL"] = "0"
# seleniumLogger.setLevel(logging.WARNING)

# Downloader

In [3]:
logger = logging.getLogger(__name__)

In [102]:
class StockDownloader(object):
    def __init__(self, timeout: int = 2):
        self.timeout = timeout
        if len(glob.glob(f"{self.download_path}/*.csv")) == 0:
            self.download_past_two_years()

    def download_data_for_date(self, date: Date, replace=False):
        download_url = self.make_url_func(date)
        file_name = date.format("YYYYMMDD") + ".csv"
        if file_name not in os.listdir(self.download_path) or replace:
            try:
                r = requests.get(
                    download_url,
                    allow_redirects=True,
                    timeout=self.timeout,
                    headers={"User-Agent": "firefox"},
                )
                r.raise_for_status()

                if self.exchange == "NSE":
                    fd, name = tempfile.mkstemp(suffix=".zip")
                    with open(name, "wb") as f:
                        f.write(r.content)
                    zipdata = zipfile.ZipFile(name)
                    zipinfos = zipdata.infolist()
                    for zipinfo in zipinfos:
                        zipinfo.filename = file_name
                        zipdata.extract(zipinfo, self.download_path)
                else:
                    with open(self.download_path / file_name, "wb") as f:
                        f.write(r.content)

                remap = {
                    "TckrSymb": "symbol",
                    "ISIN": "isin",
                    "Src": "exchange",
                    "OpnPric": "open",
                    "HghPric": "high",
                    "LwPric": "low",
                    "ClsPric": "close",
                    "TtlTradgVol": "volume",
                }
                col_order = [
                    "symbol",
                    "isin",
                    "exchange",
                    "date",
                    "open",
                    "high",
                    "low",
                    "close",
                    "volume",
                    "year",
                    "month",
                    "day",
                    "ym",
                ]

                cond = (
                    'sctysrs=="EQ"'
                    if self.exchange == "NSE"
                    else '~sctysrs.isin(["E", "F", "G", "MT"])'
                )

                df = (
                    pd.read_csv(self.download_path / file_name)
                    # .pipe(self.reformat)
                    .rename(columns=remap)
                    .assign(
                        date=date.date(),
                        year=date.year,
                        month=date.month,
                        day=date.day,
                        ym=f"{date.year}{date.month:02}",
                    )
                    .rename(columns=str.lower)
                    .query(cond)
                    .loc[:, col_order]
                )
                df.to_csv(self.download_path / file_name, index=False)
                logger.info(
                    f"Downloaded {self.exchange} data for {date.format('DD MMM, YYYY.')}"
                )

            # except HTTPError as err:
            #     if err.response.status_code == 404:
            #         logger.info(
            #             f"No {self.exchange} data available on {date.format('DD MMM, YYYY.')}"
            #         )
            # except (ReadTimeout, Timeout) as err:
            #     logger.info(
            #         f"No {self.exchange} data available on {date.format('DD MMM, YYYY.')}"
            #     )
            # except Exception as err:
            #     logger.warning(
            #         f"{self.exchange} data not available on {date.format('DD MMM, YYYY.')}"
            #     )
            #     logger.warning(err)
            finally:
                if self.exchange == "NSE":
                    os.close(fd)
        else:
            logger.info(
                f"{self.exchange} data for {date.format('DD MMM, YYYY.')} already present"
            )

    def prune_data(self, prune_weeks):
        thresh = int(today().subtract(weeks=prune_weeks).format("YYYYMMDD"))
        files_to_prune = [
            self.download_path / f"{d}.csv" for d in self.days_present if d < thresh
        ]
        for file in files_to_prune:
            os.remove(file)

    @property
    def days_present(self):
        return [
            int(d.replace(".csv", "")) for d in glob.glob1(self.download_path, "*.csv")
        ]

    def update_data(self, prune_weeks=0):
        start_date = pendulum.from_format(str(max(self.days_present)), "YYYYMMDD").add(
            days=1
        )
        if start_date < today():
            self.download_date_range(start_date, today())
        if prune_weeks:
            self.prune_data(prune_weeks)

    def download_date_range(self, start_date: Date, end_date: Date):
        assert start_date < end_date, "Start must be before end"
        dates = pd.date_range(start_date.date(), end_date.date(), freq="B").tolist()
        dates = [
            pendulum.DateTime(d.date().year, d.date().month, d.date().day)
            for d in dates
            if not d.strftime("%Y%m%d") in self.exclude_days
        ]
        for dt in dates:
            self.download_data_for_date(dt)

        # Multiprocessing seems to be causing issues and might be overkill anyway considering bot is going
        # to be running on a daily basis.
        # with ProcessPoolExecutor() as executor:
        #     executor.map(self.download_data_for_date, dates)

    def download_past_two_years(self):
        self.download_date_range(today().subtract(years=2), today())

    def download_last_n_weeks(self, n_weeks):
        self.download_date_range(today().subtract(weeks=n_weeks), today())

In [103]:
class NseDownloader(StockDownloader):
    download_path = Path("data/nse")
    exchange = "NSE"
    exclude_days = []

    def make_url_func(self, date: Date):
        date_str = date.format("YYYYMMDD").upper()
        return f"https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_{date_str}_F_0000.csv.zip"

    def reformat(self, df):
        return df.rename(columns={"TOTTRDQTY": "volume"}).assign(exchange=self.exchange)

In [104]:
class BseDownloader(StockDownloader):
    download_path = Path("data/bse")
    exchange = "BSE"
    exclude_days = ["20211229"]

    def make_url_func(self, date: Date):
        date_str = date.format("YYYYMMDD").upper()
        return f"https://www.bseindia.com/download/BhavCopy/Equity/BhavCopy_BSE_CM_0_0_0_{date_str}_F_0000.CSV"

    def reformat(self, df):
        return df.rename(
            columns={
                "NO_OF_SHRS": "volume",
                "SC_NAME": "symbol",
                "SC_CODE": "isin",
                "SC_TYPE": "series",
            }
        ).assign(
            exchange=self.exchange,
            series=lambda df: np.where(df.series == "Q", "EQ", df.series),
        )

In [105]:
nse_downloader = NseDownloader()
bse_downloader = BseDownloader()

In [106]:
nse_downloader.download_data_for_date(pendulum.DateTime(2024, 7, 26), replace=True)

In [107]:
bse_downloader.download_data_for_date(pendulum.DateTime(2024, 7, 26), replace=True)

In [ ]:
nse_downloader.update_data(prune_weeks=80)
bse_downloader.update_data(prune_weeks=80)

# Filters

In [6]:
class DataFilters(object):
    def __init__(self, as_of_date):
        df_all = pd.concat(
            map(pd.read_csv, glob.glob("data/nse/*.csv") + glob.glob("data/bse/*.csv"))
        ).assign(
            date=lambda df: pd.to_datetime(df.date),
            quarter=lambda df: df.date.dt.quarter,
        )
        self.as_of_date = pd.to_datetime(as_of_date.naive())
        max_date = df_all.date.max()
        self.filter_date = max_date if max_date < self.as_of_date else self.as_of_date
        self.df_all = df_all.query("date <= @self.filter_date")
        self.date_str = f"{self.filter_date.year}{self.filter_date.month:02}{self.filter_date.day:02}"
        logging.info(
            f"Filtering as of {self.date_str} (Either the date provided or the latest one available). "
        )

    def get_prev_data(self):
        quarter_start = self.current_quarter_start(self.filter_date).date()
        quarter_end = self.filter_date.date() - pendulum.duration(days=1)
        month_start = self.filter_date.replace(day=1).date()
        month_end = self.filter_date.date() - pendulum.duration(days=1)
        quarter_period = pendulum.period(quarter_start, quarter_end)
        month_period = pendulum.period(month_start, month_end)
        date_strs = [
            f"{dt.year}{dt.month:02}{dt.day:02}" for dt in quarter_period.range("days")
        ]
        month_days = [
            int(f"{dt.year}{dt.month:02}{dt.day:02}")
            for dt in month_period.range("days")
        ]
        prev_days = [
            f"data/filtered/{d}.xlsx"
            for d in date_strs
            if f"data/filtered/{d}.xlsx" in glob.glob("data/filtered/*.xlsx")
        ]
        self.df_prev = (
            pd.concat(map(pd.read_excel, prev_days)).assign(
                is_month=lambda df: df.date_str.isin(month_days)
            )
            if prev_days
            else None
        )

    def apply_all_filters(self):
        self.get_prev_data()
        self.apply_300p_month_filter()
        self.apply_200p_quarter_filter()
        self.apply_200p_twice_6mos()
        self.apply_52week_high_filter()
        self.df_all_filtered = (
            pd.concat(
                [
                    self.df_300p_val_month.assign(filter="300% value over prior month"),
                    self.df_200p_val_quarter.assign(
                        filter="200% value over prior quarter"
                    ),
                    self.df_52_week_highs.assign(filter="52 week high"),
                    self.df_200p_val_twice.assign(filter="200% twice in 6 months"),
                ]
            )
            .groupby(["symbol", "isin", "exchange"])
            .agg({"filter": lambda x: x.str.cat(sep=", ")})
            .reset_index()
            .assign(date_str=self.date_str)
        )
        self.df_all_filtered.to_excel(
            f"data/filtered/{self.date_str}.xlsx", index=False
        )
        logging.info(
            f"Exported results to data/filtered/{self.date_str}.xlsx. There are {self.df_all_filtered.shape[0]} scripts to scrape."
        )

    def current_quarter_start(self, ref):
        if ref.month < 4:
            return pendulum.DateTime(ref.year, 1, 1)
        elif ref.month < 7:
            return pendulum.DateTime(ref.year, 4, 1)
        elif ref.month < 10:
            return pendulum.DateTime(ref.year, 7, 1)
        return pendulum.DateTime(ref.year, 10, 1)

    def apply_300p_month_filter(self):
        prev_month_first = (self.filter_date - pd.DateOffset(months=1)).replace(day=1)
        grouping_vars = ["symbol", "isin", "exchange", "year", "month"]
        self.df_300p_val_month = (
            self.df_all.query("date >= @prev_month_first")
            .assign(value=lambda df: df.close * df.volume)
            .sort_values(grouping_vars + ["day"])
            .groupby(grouping_vars)
            .agg({"value": sum, "volume": sum, "close": lambda x: x.iloc[-1]})
            .reset_index()
            .sort_values(grouping_vars)
            .assign(
                value_lag=lambda df: df.groupby(["symbol", "isin"])["value"].shift(1),
                volume_lag=lambda df: df.groupby(["symbol", "isin"])["volume"].shift(1),
                close_lag=lambda df: df.groupby(["symbol", "isin"])["close"].shift(1),
                value_ratio=lambda df: df.value / df.value_lag,
                volume_ratio=lambda df: df.volume / df.volume_lag,
                close_ratio=lambda df: df.close / df.close_lag,
            )
            .query("value_lag.notna()", engine="python")
            .query("value_ratio>3 & close_ratio>1 & value>2_000_000")
        )
        if self.df_prev is not None:
            self.df_300p_val_month = (
                self.df_300p_val_month.merge(
                    self.df_prev.query(
                        "filter.str.contains('300% value over prior month') & is_month",
                        engine="python",
                    )
                    .loc[:, ["symbol", "isin", "exchange"]]
                    .assign(already_exists=True),
                    "left",
                )
                .query("already_exists.isna()", engine="python")
                .drop(columns="already_exists")
            )

    def previous_quarter_start(self, ref):
        if ref.month < 4:
            return pendulum.DateTime(ref.year - 1, 10, 1)
        elif ref.month < 7:
            return pendulum.DateTime(ref.year, 1, 1)
        elif ref.month < 10:
            return pendulum.DateTime(ref.year, 4, 1)
        return pendulum.DateTime(ref.year, 7, 1)

    def apply_200p_quarter_filter(self):
        prev_quarter_first = pd.to_datetime(
            self.previous_quarter_start(self.filter_date)
        )
        grouping_vars = ["symbol", "isin", "exchange", "year", "quarter"]
        self.df_200p_val_quarter = (
            self.df_all.query("date >= @prev_quarter_first")
            .assign(value=lambda df: df.close * df.volume)
            .sort_values(grouping_vars + ["month", "day"])
            .groupby(grouping_vars)
            .agg({"value": sum, "volume": sum, "close": lambda x: x.iloc[-1]})
            .reset_index()
            .sort_values(grouping_vars)
            .assign(
                value_lag=lambda df: df.groupby(["symbol", "isin"])["value"].shift(1),
                volume_lag=lambda df: df.groupby(["symbol", "isin"])["volume"].shift(1),
                close_lag=lambda df: df.groupby(["symbol", "isin"])["close"].shift(1),
                value_ratio=lambda df: df.value / df.value_lag,
                volume_ratio=lambda df: df.volume / df.volume_lag,
                close_ratio=lambda df: df.close / df.close_lag,
            )
            .query("value_lag.notna()", engine="python")
            .query("value_ratio>2 & close_ratio>1 & value>6_000_000")
        )
        if self.df_prev is not None:
            self.df_200p_val_quarter = (
                self.df_200p_val_quarter.merge(
                    self.df_prev.query(
                        "filter.str.contains('200% value over prior quarter')",
                        engine="python",
                    )
                    .loc[:, ["symbol", "isin", "exchange"]]
                    .assign(already_exists=True),
                    "left",
                )
                .query("already_exists.isna()", engine="python")
                .drop(columns="already_exists")
            )

    def apply_52week_high_filter(self):
        date_52_weeks_prior = self.filter_date - pd.DateOffset(weeks=52)
        grouping_vars = ["symbol", "isin", "exchange"]

        df_52_week_high_vals = (
            self.df_all.query("date >= @date_52_weeks_prior")
            .sort_values(grouping_vars + ["year", "month", "day"])
            .groupby(grouping_vars)
            .agg({"high": max})
        )

        self.df_52_week_highs = (
            self.df_all.query("date == @self.filter_date")
            .merge(
                df_52_week_high_vals,
                how="left",
                on=grouping_vars,
                suffixes=("", "_max"),
            )
            .query("high==high_max")
            .drop(columns="high_max")
        )

    def apply_200p_twice_6mos(self):
        date_6mos_prior = (self.filter_date - pd.DateOffset(months=6)).replace(day=1)
        grouping_vars = ["symbol", "isin", "exchange", "year", "month"]
        df_200p_val_twice_filter = (
            self.df_all.query("date >= @date_6mos_prior")
            .assign(value=lambda df: df.close * df.volume)
            .sort_values(grouping_vars + ["day"])
            .groupby(grouping_vars)
            .agg({"value": sum, "volume": sum, "close": lambda x: x.iloc[-1]})
            .reset_index()
            .sort_values(grouping_vars)
            .assign(
                value_lag=lambda df: df.groupby(["symbol", "isin"])["value"].shift(1),
                volume_lag=lambda df: df.groupby(["symbol", "isin"])["volume"].shift(1),
                close_lag=lambda df: df.groupby(["symbol", "isin"])["close"].shift(1),
                value_ratio=lambda df: df.value / df.value_lag,
                volume_ratio=lambda df: df.volume / df.volume_lag,
                close_ratio=lambda df: df.close / df.close_lag,
            )
            .query("value_lag.notna()", engine="python")
            .query("value_ratio>2 & close_ratio>1 & value>2_000_000")
            .groupby(["symbol", "isin", "exchange"])
            .agg({"close_ratio": "count"})
            .reset_index()
            .rename(columns={"close_ratio": "n_double"})
            .query("n_double>=2")
            .drop(columns="n_double")
        )
        self.df_200p_val_twice = self.df_all.query("date == @self.filter_date").merge(
            df_200p_val_twice_filter, how="inner"
        )

        if self.df_prev is not None:
            self.df_200p_val_twice = (
                self.df_200p_val_twice.merge(
                    self.df_prev.query(
                        "filter.str.contains('200% twice in 6 months') & is_month",
                        engine="python",
                    )
                    .loc[:, ["symbol", "isin", "exchange"]]
                    .assign(already_exists=True),
                    "left",
                )
                .query("already_exists.isna()", engine="python")
                .drop(columns="already_exists")
            )

    def __repr__(self):
        return f"DateFilter({self.date_str})"

In [7]:
# for d in range(1, 26):
#     as_of_date = pendulum.from_format(f"202202{d:02}", "YYYYMMDD")  # pendulum.today()
#     data_filter = DataFilters(as_of_date)
#     data_filter.apply_all_filters()

In [8]:
as_of_date = pendulum.today()  # pendulum.from_format(f"202202{d:02}", "YYYYMMDD")
data_filter = DataFilters(as_of_date)
data_filter.apply_all_filters()

# Scraping

In [9]:
class PageFinder(object):
    """ """

    base_search_url = "http://www.moneycontrol.com/stocks/cptmarket/compsearchnew.php?topsearch_type=1&search_str="

    keys_dict = pd.read_csv("data/keys.csv").set_index("field").to_dict(orient="index")
    keys_dict = {k: v["identifier"] for k, v in keys_dict.items()}
    check_element = keys_dict["market_cap"].replace(".", "")

    def __init__(self, isin, symbol):
        self.isin, self.symbol = str(isin), str(symbol)
        self.url = self.home_content = self.ratios_url = None
        self.props = dict()

        options = webdriver.ChromeOptions()
        options.add_argument("--ignore-certificate-errors")
        options.add_argument("--incognito")
        options.add_argument("--headless")
        options.add_experimental_option("excludeSwitches", ["enable-logging"])
        self.browser = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()), options=options
        )
        self.try_finding_info()

    def make_url(self, term):
        return self.base_search_url + term

    def get_parsed_content(self, url, wait_time=0):
        self.browser.implicitly_wait(wait_time)
        self.browser.get(url)
        return BeautifulSoup(self.browser.page_source)

    def validate_and_gather_info(self, term):
        url = self.make_url(term)
        content = requests.get(url).content
        if self.check_element in str(content):
            logging.info(f"Gathering Data for {self.symbol}")
            self.url = url
            soup_home = self.get_parsed_content(url, 5)
            market_cap = soup_home.select_one(
                self.keys_dict["market_cap"].replace(" ", ".")
            ).text.replace(",", "")
            self.props["market_cap"] = float(market_cap)
            #             self.props["BLANK"] = ""

            self.get_ratios(soup_home)
            self.get_financials()
            return True
        else:
            return False

    def parse_series(self, content, selector):
        ls = []
        for x in content.select(self.keys_dict[selector]):
            try:
                f = float(x.text.replace(",", ""))
                ls.append(f)
            except:
                ls.append(None)
        if len(ls) < 5:
            ls += [None] * (5 - len(ls))
        return ls

    def get_ratios(self, soup):
        self.standalone_ratios_url = soup.select_one(self.keys_dict["ratios_url"])[
            "href"
        ]
        self.consolidated_ratios_url = self.standalone_ratios_url.replace(
            "ratiosVI", "consolidated-ratiosVI"
        )
        for name, url in zip(
            ["consolidated", "standalone"],
            [self.consolidated_ratios_url, self.standalone_ratios_url],
        ):
            content = self.get_parsed_content(url)
            for metric in ["rnw", "de"]:
                self.props[f"{name}_{metric}"] = self.parse_series(
                    content, f"{name}_{metric}"
                )

    def get_financials(self):
        self.standlone_financials_url = self.standalone_ratios_url.replace(
            "ratiosVI", "results/yearly"
        )
        self.consolidated_financials_url = self.consolidated_ratios_url.replace(
            "consolidated-ratiosVI", "results/consolidated-yearly"
        )

        for name, url in zip(
            ["consolidated", "standalone"],
            [self.consolidated_financials_url, self.standlone_financials_url],
        ):
            content = self.get_parsed_content(url)
            for metric in ["sr", "np"]:
                self.props[f"{name}_{metric}"] = self.parse_series(
                    content, f"{name}_{metric}"
                )

    def try_finding_info(self):
        search_terms = [self.isin] + [
            self.symbol[:i] for i in range(len(self.symbol), 3, -1)
        ]
        for term in search_terms:
            if self.validate_and_gather_info(term):
                logging.info(f"Found data for {self.symbol}")
                return
        logging.warning(f"Could not find data for {self.symbol}")
        self.browser.close()

    def __repr__(self):
        return f"PageFinder({self.isin}, {self.symbol}, {self.url})"

In [10]:
df_filtered = data_filter.df_all_filtered

In [12]:
# def scrape_metrics(df_filtered):
df_filtered = df_filtered.iloc[:5].assign(
    pf=lambda df: df.progress_apply(
        lambda row: PageFinder(row["isin"], row["symbol"]), axis=1
    ),
)

100%|████████████████████████████████████████████| 5/5 [01:24<00:00, 16.89s/it]


In [13]:
df_filtered

,symbol,isin,exchange,filter,date_str,pf
0,ABHIFIN,538935,BSE,52 week high,20220225,"PageFinder(538935, ABHIFIN , http://www.mo..."
1,ABMTSPRG,542887,BSE,52 week high,20220225,"PageFinder(542887, ABMTSPRG , None)"
2,ACTIVE,541144,BSE,52 week high,20220225,"PageFinder(541144, ACTIVE , http://www.mo..."
3,AMANI TRAD,512008,BSE,52 week high,20220225,"PageFinder(512008, AMANI TRAD , http://www.mo..."
4,AVIVA INDS,512109,BSE,52 week high,20220225,"PageFinder(512109, AVIVA INDS , http://www.mo..."


In [15]:
df = (
    df_filtered.pf.apply(lambda pf: pf.props)
    .apply(pd.Series)
    .query("market_cap.notna()", engine="python")
)
cols = [
    "consolidated_rnw",
    "standalone_rnw",
    "consolidated_sr",
    "consolidated_np",
    "standalone_sr",
    "standalone_np",
    "consolidated_de",
    "standalone_de",
]
for b, col in enumerate(cols):
    names = [f"{col.replace('_', ' ')}{i}" for i in range(5, 0, -1)]
    df[names] = pd.DataFrame(df[col].tolist(), index=df.index)
    df[f"BLANK {b}"] = ""
df = df.drop(columns=cols)

In [17]:
df

,market_cap,consolidated rnw5,consolidated rnw4,consolidated rnw3,consolidated rnw2,consolidated rnw1,BLANK 0,standalone rnw5,standalone rnw4,standalone rnw3,standalone rnw2,standalone rnw1,BLANK 1,consolidated sr5,consolidated sr4,consolidated sr3,consolidated sr2,consolidated sr1,BLANK 2,consolidated np5,consolidated np4,consolidated np3,consolidated np2,consolidated np1,BLANK 3,standalone sr5,standalone sr4,standalone sr3,standalone sr2,standalone sr1,BLANK 4,standalone np5,standalone np4,standalone np3,standalone np2,standalone np1,BLANK 5,consolidated de5,consolidated de4,consolidated de3,consolidated de2,consolidated de1,BLANK 6,standalone de5,standalone de4,standalone de3,standalone de2,standalone de1,BLANK 7
0,0.0,None,None,None,None,None,,2.09,-2.23,1.00,1.59,1.54,,None,None,None,None,None,,None,None,None,None,None,,0.51,0.33,0.71,3.01,0.47,,0.08,-0.09,0.03,0.05,0.05,,None,None,None,None,None,,0.02,0.02,0.00,0.00,0.00,
2,44.0,None,None,None,None,None,,0.08,1.39,5.03,4.15,5.43,,None,None,None,None,None,,None,None,None,None,None,,73.14,147.96,154.49,127.35,119.98,,0.06,0.86,3.06,2.37,1.86,,None,None,None,None,None,,0.87,0.99,0.99,0.90,2.08,
3,8.0,None,None,None,None,None,,4.34,9.46,-1.73,5.02,3.13,,None,None,None,None,None,,None,None,None,None,None,,NaN,0.04,2.13,2.84,5.36,,0.07,0.15,-0.02,0.07,0.03,,None,None,None,None,None,,0.00,0.00,0.00,0.00,0.00,
4,2.0,None,None,None,None,None,,0.07,0.03,1.90,0.56,0.15,,None,None,None,None,None,,None,None,None,None,None,,0.10,NaN,NaN,1.36,NaN,,0.00,0.00,0.08,0.02,0.01,,None,None,None,None,None,,0.04,0.04,0.04,0.02,0.04,


In [20]:
df_filtered[["symbol", "isin", "exchange", "filter"]]

,symbol,isin,exchange,filter
0,ABHIFIN,538935,BSE,52 week high
1,ABMTSPRG,542887,BSE,52 week high
2,ACTIVE,541144,BSE,52 week high
3,AMANI TRAD,512008,BSE,52 week high
4,AVIVA INDS,512109,BSE,52 week high


In [21]:
df

,market_cap,consolidated rnw5,consolidated rnw4,consolidated rnw3,consolidated rnw2,consolidated rnw1,BLANK 0,standalone rnw5,standalone rnw4,standalone rnw3,standalone rnw2,standalone rnw1,BLANK 1,consolidated sr5,consolidated sr4,consolidated sr3,consolidated sr2,consolidated sr1,BLANK 2,consolidated np5,consolidated np4,consolidated np3,consolidated np2,consolidated np1,BLANK 3,standalone sr5,standalone sr4,standalone sr3,standalone sr2,standalone sr1,BLANK 4,standalone np5,standalone np4,standalone np3,standalone np2,standalone np1,BLANK 5,consolidated de5,consolidated de4,consolidated de3,consolidated de2,consolidated de1,BLANK 6,standalone de5,standalone de4,standalone de3,standalone de2,standalone de1,BLANK 7
0,0.0,None,None,None,None,None,,2.09,-2.23,1.00,1.59,1.54,,None,None,None,None,None,,None,None,None,None,None,,0.51,0.33,0.71,3.01,0.47,,0.08,-0.09,0.03,0.05,0.05,,None,None,None,None,None,,0.02,0.02,0.00,0.00,0.00,
2,44.0,None,None,None,None,None,,0.08,1.39,5.03,4.15,5.43,,None,None,None,None,None,,None,None,None,None,None,,73.14,147.96,154.49,127.35,119.98,,0.06,0.86,3.06,2.37,1.86,,None,None,None,None,None,,0.87,0.99,0.99,0.90,2.08,
3,8.0,None,None,None,None,None,,4.34,9.46,-1.73,5.02,3.13,,None,None,None,None,None,,None,None,None,None,None,,NaN,0.04,2.13,2.84,5.36,,0.07,0.15,-0.02,0.07,0.03,,None,None,None,None,None,,0.00,0.00,0.00,0.00,0.00,
4,2.0,None,None,None,None,None,,0.07,0.03,1.90,0.56,0.15,,None,None,None,None,None,,None,None,None,None,None,,0.10,NaN,NaN,1.36,NaN,,0.00,0.00,0.08,0.02,0.01,,None,None,None,None,None,,0.04,0.04,0.04,0.02,0.04,


In [19]:
pd.concat(
    [
        df_filtered[["symbol", "isin", "exchange", "filter"]],
        df,
        df_filtered[["date_str"]],
    ],
    axis=1,
)

,symbol,isin,exchange,filter,market_cap,consolidated rnw5,consolidated rnw4,consolidated rnw3,consolidated rnw2,consolidated rnw1,BLANK 0,standalone rnw5,standalone rnw4,standalone rnw3,standalone rnw2,standalone rnw1,BLANK 1,consolidated sr5,consolidated sr4,consolidated sr3,consolidated sr2,consolidated sr1,BLANK 2,consolidated np5,consolidated np4,consolidated np3,consolidated np2,consolidated np1,BLANK 3,standalone sr5,standalone sr4,standalone sr3,standalone sr2,standalone sr1,BLANK 4,standalone np5,standalone np4,standalone np3,standalone np2,standalone np1,BLANK 5,consolidated de5,consolidated de4,consolidated de3,consolidated de2,consolidated de1,BLANK 6,standalone de5,standalone de4,standalone de3,standalone de2,standalone de1,BLANK 7,date_str
0,ABHIFIN,538935,BSE,52 week high,0.0,None,None,None,None,None,,2.09,-2.23,1.00,1.59,1.54,,None,None,None,None,None,,None,None,None,None,None,,0.51,0.33,0.71,3.01,0.47,,0.08,-0.09,0.03,0.05,0.05,,None,None,None,None,None,,0.02,0.02,0.00,0.00,0.00,,20220225
1,ABMTSPRG,542887,BSE,52 week high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220225
2,ACTIVE,541144,BSE,52 week high,44.0,None,None,None,None,None,,0.08,1.39,5.03,4.15,5.43,,None,None,None,None,None,,None,None,None,None,None,,73.14,147.96,154.49,127.35,119.98,,0.06,0.86,3.06,2.37,1.86,,None,None,None,None,None,,0.87,0.99,0.99,0.90,2.08,,20220225
3,AMANI TRAD,512008,BSE,52 week high,8.0,None,None,None,None,None,,4.34,9.46,-1.73,5.02,3.13,,None,None,None,None,None,,None,None,None,None,None,,NaN,0.04,2.13,2.84,5.36,,0.07,0.15,-0.02,0.07,0.03,,None,None,None,None,None,,0.00,0.00,0.00,0.00,0.00,,20220225
4,AVIVA INDS,512109,BSE,52 week high,2.0,None,None,None,None,None,,0.07,0.03,1.90,0.56,0.15,,None,None,None,None,None,,None,None,None,None,None,,0.10,NaN,NaN,1.36,NaN,,0.00,0.00,0.08,0.02,0.01,,None,None,None,None,None,,0.04,0.04,0.04,0.02,0.04,,20220225


In [15]:
df_final = (
    pd.concat(
        [
            df_filtered[["symbol", "isin", "exchange", "filter"]],
            df,
            df_filtered[["date_str"]],
        ],
        axis=1,
    )
    .assign(no_data=lambda df: df.market_cap.isna())
    .sort_values("no_data")
)

df_final.loc[~df_final.no_data] = df_final.loc[~df_final.no_data].fillna("")
df_final.loc[df_final.no_data] = df_final.loc[df_final.no_data].fillna(
    "No Match in Money Control"
)
df_final = df_final.drop(columns="no_data").rename(columns=str.upper)

# # Load the file
# wb = openpyxl.load_workbook("data/results_template.xlsx")
# ws = wb.active

# # Convert the dataframe into rows
# rows = dataframe_to_rows(df_final, index=False, header=False)

# # Write the rows to the worksheet
# for r_idx, row in enumerate(rows, 2):
#     for c_idx, value in enumerate(row, 1):
#         ws.cell(row=r_idx, column=c_idx, value=value)

# # Save the worksheet as a (*.xlsx) file
# wb.template = False
# wb.save(f"data/final/{data_filter.date_str}.xlsx")

In [16]:
df_final

,SYMBOL,ISIN,EXCHANGE,FILTER,MARKET_CAP,CONSOLIDATED RNW5,CONSOLIDATED RNW4,CONSOLIDATED RNW3,CONSOLIDATED RNW2,CONSOLIDATED RNW1,BLANK 0,STANDALONE RNW5,STANDALONE RNW4,STANDALONE RNW3,STANDALONE RNW2,STANDALONE RNW1,BLANK 1,CONSOLIDATED SR5,CONSOLIDATED SR4,CONSOLIDATED SR3,CONSOLIDATED SR2,CONSOLIDATED SR1,BLANK 2,CONSOLIDATED NP5,CONSOLIDATED NP4,CONSOLIDATED NP3,CONSOLIDATED NP2,CONSOLIDATED NP1,BLANK 3,STANDALONE SR5,STANDALONE SR4,STANDALONE SR3,STANDALONE SR2,STANDALONE SR1,BLANK 4,STANDALONE NP5,STANDALONE NP4,STANDALONE NP3,STANDALONE NP2,STANDALONE NP1,BLANK 5,CONSOLIDATED DE5,CONSOLIDATED DE4,CONSOLIDATED DE3,CONSOLIDATED DE2,CONSOLIDATED DE1,BLANK 6,STANDALONE DE5,STANDALONE DE4,STANDALONE DE3,STANDALONE DE2,STANDALONE DE1,BLANK 7,DATE_STR
0,ABHIFIN,538935,BSE,52 week high,0.0,,,,,,,2.09,-2.23,1.0,1.59,1.54,,,,,,,,,,,,,,0.51,0.33,0.71,3.01,0.47,,0.08,-0.09,0.03,0.05,0.05,,,,,,,,0.02,0.02,0.0,0.0,0.0,,20220225
2,ACTIVE,541144,BSE,52 week high,44.0,,,,,,,0.08,1.39,5.03,4.15,5.43,,,,,,,,,,,,,,73.14,147.96,154.49,127.35,119.98,,0.06,0.86,3.06,2.37,1.86,,,,,,,,0.87,0.99,0.99,0.9,2.08,,20220225
3,AMANI TRAD,512008,BSE,52 week high,8.0,,,,,,,4.34,9.46,-1.73,5.02,3.13,,,,,,,,,,,,,,,0.04,2.13,2.84,5.36,,0.07,0.15,-0.02,0.07,0.03,,,,,,,,0.0,0.0,0.0,0.0,0.0,,20220225
4,AVIVA INDS,512109,BSE,52 week high,2.0,,,,,,,0.07,0.03,1.9,0.56,0.15,,,,,,,,,,,,,,0.1,,,1.36,,,0.0,0.0,0.08,0.02,0.01,,,,,,,,0.04,0.04,0.04,0.02,0.04,,20220225
1,ABMTSPRG,542887,BSE,52 week high,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,No Match in Money Control,20220225


Deal with 

1. ~Unequal number of elements for metrics~
2. Stock not found. Have an exception  url csv? 
3. ~Wait for page to load to get elementslike market cap~
4. ~Add the right css codes to scrape~
4. Formatting - freeze panes, stripes, tables, sort by absence,

In [230]:
df

,market_cap,consolidated rnw5,consolidated rnw4,consolidated rnw3,consolidated rnw2,consolidated rnw1,BLANK 0,standalone rnw5,standalone rnw4,standalone rnw3,standalone rnw2,standalone rnw1,BLANK 1,consolidated sr5,consolidated sr4,consolidated sr3,consolidated sr2,consolidated sr1,BLANK 2,consolidated np5,consolidated np4,consolidated np3,consolidated np2,consolidated np1,BLANK 3,standalone sr5,standalone sr4,standalone sr3,standalone sr2,standalone sr1,BLANK 4,standalone np5,standalone np4,standalone np3,standalone np2,standalone np1,BLANK 5,consolidated de5,consolidated de4,consolidated de3,consolidated de2,consolidated de1,BLANK 6,standalone de5,standalone de4,standalone de3,standalone de2,standalone de1,BLANK 7
0,10.0,None,None,None,None,None,,2.09,-2.23,1.00,1.59,1.54,,None,None,None,None,None,,None,None,None,None,None,,0.51,0.33,0.71,3.01,0.47,,0.08,-0.09,0.03,0.05,0.05,,None,None,None,None,None,,0.02,0.02,0.00,0.0,0.00,
2,44.0,None,None,None,None,None,,0.08,1.39,5.03,4.15,5.43,,None,None,None,None,None,,None,None,None,None,None,,73.14,147.96,154.49,127.35,119.98,,0.06,0.86,3.06,2.37,1.86,,None,None,None,None,None,,0.87,0.99,0.99,0.9,2.08,


In [257]:
shutil.copy("data/results_template.xlsx", f"data/final/{data_filter.date_str}.xlsx")

'data/final/20220225.xlsx'

<Cell 'Sheet1'.A2>

<Cell 'Sheet1'.B2>

<Cell 'Sheet1'.C2>

<Cell 'Sheet1'.D2>

<Cell 'Sheet1'.E2>

...

<Cell 'Sheet1'.AX4>

<Cell 'Sheet1'.AY4>

<Cell 'Sheet1'.AZ4>

<Cell 'Sheet1'.BA4>

<Cell 'Sheet1'.BB4>

# Testing & Validation

In [32]:
# df_filtered_invalid = df_filtered.query("url.isna()", engine="python")
# df_filtered_valid = df_filtered.query("url.notna()", engine="python")
# df_filtered_invalid.to_excel(f"data/filtered/df_invalid_{date_str}.xlsx")
# df_filtered_valid.to_excel(f"data/filtered/df_valid_{date_str}.xlsx")
# df_filtered_invalid.to_excel("data/df_filtered_invalid.xlsx")
# df_filtered_valid.to_excel("data/df_filtered_valid.xlsx")

In [86]:
df_all.query("isin==504084")

,symbol,isin,exchange,date,open,high,low,close,volume,year,month,day,ym,quarter
353,KAYCEE IND,504084,BSE,2021-08-06,4000.0,4170.00,3936.00,3938.40,34,2021,8,6,202108,3
351,KAYCEE IND,504084,BSE,2021-07-23,4393.6,4393.60,4151.00,4151.00,7,2021,7,23,202107,3
350,KAYCEE IND,504084,BSE,2021-08-31,3935.0,3935.00,3745.05,3745.05,11,2021,8,31,202108,3
356,KAYCEE IND,504084,BSE,2021-10-05,3703.0,4040.00,3701.00,4040.00,29,2021,10,5,202110,4
344,KAYCEE IND,504084,BSE,2020-12-17,2960.0,2960.00,2750.00,2750.00,11,2020,12,17,202012,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,KAYCEE IND,504084,BSE,2021-08-05,4075.0,4076.95,4050.00,4050.00,6,2021,8,5,202108,3
352,KAYCEE IND,504084,BSE,2021-09-29,3851.0,4044.00,3851.00,4044.00,4,2021,9,29,202109,3
358,KAYCEE IND,504084,BSE,2021-06-25,3849.9,3854.55,3849.90,3854.55,23,2021,6,25,202106,2
355,KAYCEE IND,504084,BSE,2021-05-17,3000.0,3044.00,2900.00,3030.00,29,2021,5,17,202105,2


In [87]:
x = ["ACCORD", "BDL", "EIDPARRY", "ELECTCAST", "SIS", "SRPL", "VINNY", "VIPCLOTHNG"]
df_all.query("symbol.isin(@x)", engine="python")[["symbol", "isin"]].drop_duplicates()

,symbol,isin
176,BDL,INE171Z01018
364,EIDPARRY,INE126A01031
369,ELECTCAST,INE086A01029
1246,SIS,INE285J01028
1474,VIPCLOTHNG,INE450G01024
1270,SRPL,INE008Z01012


In [92]:
#  .assign(value=lambda df: df.close * df.volume)
#     .sort_values(grouping_vars + ["month", "day"])
#     .groupby(grouping_vars)
#     .agg({"value": sum, "volume": sum, "close": lambda x: x.iloc[-1]})
#     .reset_index()
#     .sort_values(grouping_vars)
#     .assign(
#         value_lag=lambda df: df.groupby(["symbol", "isin"])["value"].shift(1),
#         volume_lag=lambda df: df.groupby(["symbol", "isin"])["volume"].shift(1),
#         close_lag=lambda df: df.groupby(["symbol", "isin"])["close"].shift(1),
#         value_ratio=lambda df: df.value / df.value_lag,
#         volume_ratio=lambda df: df.volume / df.volume_lag,
#         close_ratio=lambda df: df.close / df.close_lag,
#     )

In [107]:
df_tmp = (
    df_all.query("isin==502901")
    .query("date>='2021-10-01'")
    .sort_values(["year", "quarter", "month", "day"])
    .assign(value=lambda df: df.close * df.volume)
    .groupby(["year", "quarter"])
    .agg({"value": sum, "volume": sum, "close": lambda x: x.iloc[-1]})
    .reset_index()
    .assign(value_last_close=lambda df: df.close * df.volume)
)

df_tmp

print("ratios")
df_tmp.iloc[1] / df_tmp.iloc[0]

,year,quarter,value,volume,close,value_last_close
0,2021,4,1448204.15,413,4016.15,1658669.95
1,2022,1,189562.00,44,4290.00,188760.00


ratios


year                1.000495
quarter             0.250000
value               0.130895
volume              0.106538
close               1.068187
value_last_close    0.113802
dtype: float64

In [108]:
87825600 / 71640000

1.225929648241206

In [109]:
df = df_all.query("isin==502901").query("year==2022 & quarter==1").sort_values("date")
df
"volume", f"{df.volume.sum():,}"
"last close", f"{df.close.iloc[-1]:,}"
"value", f"{df.close.iloc[-1]*df.volume.sum():,}"

,symbol,isin,exchange,date,open,high,low,close,volume,year,month,day,ym,quarter
320,JAMSHRI,502901,BSE,2022-01-03,3815.5,4200.0,3815.5,4190.0,14,2022,1,3,202201,1
322,JAMSHRI,502901,BSE,2022-01-10,4390.0,4390.0,4200.0,4370.0,5,2022,1,10,202201,1
317,JAMSHRI,502901,BSE,2022-01-12,4151.5,4580.0,4151.5,4580.0,12,2022,1,12,202201,1
316,JAMSHRI,502901,BSE,2022-01-14,4580.0,4580.0,4351.0,4351.0,2,2022,1,14,202201,1
322,JAMSHRI,502901,BSE,2022-01-17,4140.0,4140.0,4140.0,4140.0,2,2022,1,17,202201,1
315,JAMSHRI,502901,BSE,2022-01-18,3933.0,3950.0,3933.0,3950.0,3,2022,1,18,202201,1
314,JAMSHRI,502901,BSE,2022-01-19,4147.0,4147.0,4147.0,4147.0,2,2022,1,19,202201,1
314,JAMSHRI,502901,BSE,2022-01-21,4300.0,4300.0,4300.0,4300.0,1,2022,1,21,202201,1
320,JAMSHRI,502901,BSE,2022-01-24,4086.0,4086.0,4086.0,4086.0,1,2022,1,24,202201,1
314,JAMSHRI,502901,BSE,2022-01-27,4290.0,4290.0,4290.0,4290.0,2,2022,1,27,202201,1


('volume', '44')

('last close', '4,290.0')

('value', '188,760.0')

## 8/1

In [5]:
import requests
from typing import Union
import os
import pandas as pd
import logging
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

# from googlesearch import search
from duckduckgo_search import DDGS

logger = logging.getLogger(__name__)
tqdm.pandas()
os.environ["WDM_LOG_LEVEL"] = "0"

In [ ]:
df_filtered = pd.read_excel("data/df_latest_filtered.xlsx").iloc[:50]
df_filtered.head()
result = next(results)

In [18]:
class PageFinder(object):
    """ """

    keys_dict = pd.read_csv("data/keys.csv").set_index("field").to_dict(orient="index")
    keys_dict = {k: v["identifier"] for k, v in keys_dict.items()}
    check_element = keys_dict["market_cap"].replace(".", "")

    def __init__(self, isin, symbol):
        self.isin, self.symbol = str(isin), str(symbol)
        self.url = self.home_content = self.ratios_url = None
        self.props = dict()
        self.try_finding_info()

    def get_parsed_content(self, url):
        return BeautifulSoup(requests.get(url).content, features="lxml")

    def validate_and_gather_info(self, symbol, isin, force="symbol"):
        print(f"symbol: {symbol}, isin: {isin}, force: {force}")
        if force == "symbol":
            search_term = f'"{symbol}" {isin} site:https://www.moneycontrol.com/india/stockpricequote/'
        elif force == "isin":
            search_term = f'{symbol} "{isin}" site:https://www.moneycontrol.com/india/stockpricequote/'
        elif force == "both":
            search_term = f'"{symbol}" "{isin}" site:https://www.moneycontrol.com/india/stockpricequote/'
        else:
            search_term = f"{symbol} {isin} site:https://www.moneycontrol.com/india/stockpricequote/"

        results = DDGS().text(
            search_term,
            max_results=5,
        )

        if not results:
            return False

        try:
            url = results[0]["href"]
            content = str(requests.get(url).content)
            assert symbol.lower() in content.lower() or isin.lower() in content.lower()
        except Exception as e:
            logger.warning(
                f"Parse error for symbol: {symbol}, isin: {isin}, force: {force}, search_term: {search_term}"
            )
            logger.warning(e)
            content = ""

        if self.check_element in content:
            logger.info(f"\nGathering Data for {self.symbol}")
            self.url = url
            soup_home = BeautifulSoup(content, features="lxml")
            market_cap = soup_home.select_one(
                self.keys_dict["market_cap"].replace(" ", ".")
            ).text.replace(",", "")
            self.props["market_cap"] = float(market_cap)
            #             self.props["BLANK"] = ""

            self.get_ratios(soup_home)
            self.get_financials()
            return True
        else:
            return False

    def parse_series(self, content, selector):
        ls = []
        for x in content.select(self.keys_dict[selector]):
            try:
                f = float(x.text.replace(",", ""))
                ls.append(f)
            except:
                ls.append(None)
        if len(ls) < 5:
            ls += [None] * (5 - len(ls))
        return ls

    def get_ratios(self, soup):
        self.standalone_ratios_url = soup.select_one(self.keys_dict["ratios_url"])[
            "href"
        ]
        self.consolidated_ratios_url = self.standalone_ratios_url.replace(
            "ratiosVI", "consolidated-ratiosVI"
        )
        for name, url in zip(
            ["consolidated", "standalone"],
            [self.consolidated_ratios_url, self.standalone_ratios_url],
        ):
            content = self.get_parsed_content(url)
            for metric in ["rnw", "de"]:
                self.props[f"{name}_{metric}"] = self.parse_series(
                    content, f"{name}_{metric}"
                )

    def get_financials(self):
        self.standlone_financials_url = self.standalone_ratios_url.replace(
            "ratiosVI", "results/yearly"
        )
        self.consolidated_financials_url = self.consolidated_ratios_url.replace(
            "consolidated-ratiosVI", "results/consolidated-yearly"
        )

        for name, url in zip(
            ["consolidated", "standalone"],
            [self.consolidated_financials_url, self.standlone_financials_url],
        ):
            content = self.get_parsed_content(url)
            for metric in ["sr", "np"]:
                self.props[f"{name}_{metric}"] = self.parse_series(
                    content, f"{name}_{metric}"
                )

    def try_finding_info(self):
        for force in ["both", "symbol", "isin", "neither"]:
            if self.validate_and_gather_info(self.symbol, self.isin, force=force):
                logger.info(f"Found data for {self.symbol}")
                return
        logger.warning(f"\nCould not find data for {self.symbol}")
        # self.browser.close()

    def __repr__(self):
        return f"PageFinder({self.isin}, {self.symbol}, {self.url})"


def scrape_metrics(df_filtered, date_str):
    df_filtered = df_filtered.assign(
        pf=lambda df: df.progress_apply(
            lambda row: PageFinder(row["isin"], row["symbol"]), axis=1
        ),
    )
    # we query only for stocks where data was successfully scraped as they are easier to split into columns
    # This works even during merging with the df_filtered because the indix is left unchanged and that makes
    # sure the alignment happens correctly.
    df = (
        df_filtered.pf.apply(lambda pf: pf.props)
        .apply(pd.Series)
        .query("market_cap.notna()", engine="python")
    )
    cols = [
        "consolidated_rnw",
        "standalone_rnw",
        "consolidated_sr",
        "consolidated_np",
        "standalone_sr",
        "standalone_np",
        "consolidated_de",
        "standalone_de",
    ]
    for b, col in enumerate(cols):
        names = [f"{col.replace('_', ' ')}{i}" for i in range(5, 0, -1)]
        df[names] = pd.DataFrame(df[col].tolist(), index=df.index)
        df[f"BLANK {b}"] = ""
    df = df.drop(columns=cols)

    # index of df and df_filtered makes sure alignement happens correctly
    df_final = (
        pd.concat(
            [
                df_filtered[["symbol", "isin", "exchange", "filter"]],
                df,
                df_filtered[["date_str"]],
            ],
            axis=1,
        )
        .assign(no_data=lambda df: df.market_cap.isna())
        .sort_values("no_data")
    )

    df_final.loc[~df_final.no_data] = df_final.loc[~df_final.no_data].fillna("")
    df_final.loc[df_final.no_data] = df_final.loc[df_final.no_data].fillna(
        "No Match in Money Control"
    )
    df_final = df_final.drop(columns="no_data").rename(columns=str.upper)

    # Load the file
    wb = openpyxl.load_workbook("data/results_template.xlsx")
    ws = wb.active

    # Convert the dataframe into rows
    rows = dataframe_to_rows(df_final, index=False, header=False)

    # Write the rows to the worksheet
    for r_idx, row in enumerate(rows, 2):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    # Save the worksheet as a (*.xlsx) file
    wb.template = False
    save_path = (
        f"data/{date_str}.xlsx"
        if date_str == "latest"
        else f"data/final/{date_str}.xlsx"
    )
    wb.save(save_path)
    logger.info(f"Saved data to {save_path}")

In [23]:
pf = PageFinder("533329", "IND TERRAIN")

symbol: IND TERRAIN, isin: 533329, force: both
symbol: IND TERRAIN, isin: 533329, force: symbol
symbol: IND TERRAIN, isin: 533329, force: isin
symbol: IND TERRAIN, isin: 533329, force: neither


In [24]:
pf.url

'https://www.moneycontrol.com/india/stockpricequote/textiles-readymade-apparels/indianterrainfashions/ITF'

In [14]:
results = DDGS().text(
    "symbol: HILTON INE788H01017 site:https://www.moneycontrol.com/india/stockpricequote/",
    max_results=5,
)
print(results)

[{'title': 'Hilton Metal Share Price, Hilton Metal Stock Price, Hilton Metal ...', 'href': 'https://www.moneycontrol.com/india/stockpricequote/castingsforgings/hiltonmetalforging/HMF', 'body': 'Hilton Metal Standalone December 2023 Net Sales at Rs 41.74 crore, up 43.74% Y-o-Y Feb 12 2024 01:12 PM Hilton Metal Standalone September 2023 Net Sales at Rs 34.81 crore, up 27.63% Y-o-Y Nov 15 ...'}, {'title': 'HIL Share Price, HIL Stock Price, HIL Ltd. Stock Price ... - Moneycontrol', 'href': 'https://www.moneycontrol.com/india/stockpricequote/cement-productsbuilding-materials/hil/HI09', 'body': 'HIL Ltd. is a leading company in the cement products and building materials sector. Find out the latest news, stock price, charts, forecasts, recommendations and more on HIL Share Price at ...'}, {'title': 'Bharti Hexacom Ltd. - Moneycontrol', 'href': 'https://www.moneycontrol.com/india/stockpricequote/miscellaneous/bhartihexacom/BHL02', 'body': 'Bharti Hexacom Share Price: Find the latest news on Bh

In [15]:
for result in results:
    if "moneycontrol" in result:
        print(result)

In [62]:
result

'https://www.moneycontrol.com/india/stockpricequote/castingsforgings/hiltonmetalforging/HMF'